## **Introduction to Logistics Regression**
### 1. Short History of Logistic Regression

The logistic function was introduced in the **19th century** by **Pierre-François Verhulst** to model population growth.

In the **early 20th century**, statisticians realized that the same function could model **binary outcomes** (yes/no, success/failure).

By the **1940s–1950s**, logistic regression was formally developed as a statistical method, especially in **biostatistics and social sciences**, to model probabilities of events that have only two possible outcomes.

**Today**, logistic regression is one of the foundational algorithms in machine learning, widely used for problems such as:

- **Churn prediction**
- **Fraud detection**
- **Medical diagnosis**

**Because it is**:
- **Interpretable**
- **Probabilistic** 
- **Mathematically well-grounded**

---

### 2. What is Logistic Regression?

**Logistic regression** is a **supervised learning algorithm** used for **binary classification**.

**Its goal**: Model the probability that an outcome belongs to the **positive class** (e.g., customer churns).

**Key difference**: Instead of predicting a class label directly, logistic regression predicts a **probability**, which is then converted into a class decision using a **threshold** (commonly **0.5**).

A natural question arises because the term **“regression”** is used, even though the task is classification. 

To understand this, we can break the idea down as follows:

- Logistic regression assumes that the **linear output produced from the features represents the log-odds (logit)** of the positive class.
- Solving this relationship yields the **sigmoid (logistic) function**, which converts the linear output into a probability.



---

### 3. Why the Linear Model Output is Not a Probability

Logistic regression starts with a **linear model**, just like linear regression:

$z = 0.8x - 1.2$


**Key issue**:

**Linear model outputs**: `z ∈ (-∞, +∞)` (any real number)

**Probabilities must satisfy**: `0 ≤ p ≤ 1`

**Therefore**:
- The linear output represents the log odds of the positive class
- The linear output tells us how strongly the evidence supports one class over the other, on a log scale
- Linear output `z` **cannot** be interpreted as a probability
- We need a transformation that maps all real numbers into the interval `(0,1)`

---

### 4. Log-Odds (Logit): Linking Linear Models to Probability

logistic regression assumes that the linear output represents the log odds of the positive class*

**Odds**:

$$odds = \frac{p}{1 - p}$$


**Log-odds (logit)**:

$$\log\left(\frac{p}{1 - p}\right)$$


**Important property**:

- Log-odds range: (-∞, +∞)
- Linear model range: (-∞, +∞)
  
✓ Perfect match!


---

### 5. Logistic Regression Model (Example)

$$\log\left(\frac{p}{1-p}\right) = 0.8x - 1.2$$



**Left-hand side**: Log-odds (logit) of churning  
**Right-hand side**: Linear regression on feature `x`

**This equation means**:
- Features influence the **log-odds of churn linearly**
- **Not** the probability directly

---

### 6. Converting Log-Odds into a Valid Probability

To recover the probability `p`, we solve the log-odds equation.

This leads to the **logistic (sigmoid) function**:

$$p = \frac{1}{1 + e^{-(0.8x - 1.2)}}$$


**Properties of the logistic function**:
- Maps **any real number** to `(0,1)`
- Produces a **smooth, interpretable probability**
- **Ensures outputs are always valid probabilities**

---

### 7. Final Interpretation

*In logistic regression, the output of the linear model is assumed to be the log-odds of the positive class. Solving this relationship gives the sigmoid (logistic) function, which maps the linear output to a probability between 0 and 1.*

1. Linear model → z = 0.8x - 1.2 (interpreted as the log-odds score)

2. Logistic function → p = sigmoid(z) (solved log-odss produces a sigmoid function which gives us a probability score)

3. Threshold → Class decision (p > 0.5 = churn)


**Complete flow**:

Features → Linear Model → Log-odds → Sigmoid → Probability → Class


**That's logistic regression!** 🎯


## **Data Preparation**

In [1]:
# import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
# convert column headers to lower case and replace space with _
# convert categorical column values to lower case and replace space with _

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [4]:
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [5]:
# convert total charges to number and replace nulls with 0
df.totalcharges = pd.to_numeric(df.totalcharges, errors = 'coerce')
df.totalcharges = df.totalcharges.fillna(0)

In [6]:
# convert yes to 1 and no to 0 and convert the datatype to int
df.churn = (df.churn == 'yes').astype(int)

## **Setting up the Validation Framework**

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
# split the dataset into train, validation and test sets
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 1)
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 1)

print(len(df_train), len(df_val), len(df_test))

4225 1409 1409


In [9]:
# reset the index
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [10]:
# seperate the target variables
y_train = df_train['churn'].values
y_val = df_val['churn'].values
y_test = df_test['churn'].values

In [11]:
# delete the target valriables in the feature matrix
del df_train['churn']
del df_val['churn']
del df_test['churn']

In [12]:
df_full_train = df_full_train.reset_index(drop=True)

In [13]:
df_full_train.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
1,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
2,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
3,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
4,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


In [14]:
# check for missing values
df_full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [15]:
# check the proportion of churn vs no churn
df_full_train.churn.value_counts(normalize = True)

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

In [16]:
global_churn_rate = round(df_full_train.churn.mean(),2)
print(global_churn_rate)

0.27


In [17]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [18]:
df_full_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [19]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
        'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [20]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

## **Feature Importance: Churn rate and risk ratio**

Feature importance analysis (part of EDA) - identifying which features affect our target

- Churn rate
- Risk ratio
- Mutual information

#### **Interpreting churn metrics**

**Difference (Group − Global):**

- If the value is less than 0, the group is less likely to churn than the global average.

- If the value is greater than 0, the group is more likely to churn than the global average.

**Risk Ratio (Group / Global):**

- If the risk ratio is greater than 1, the group is more likely to churn than the global population.

- If the risk ratio is less than 1, the group is less likely to churn than the global population.

In [21]:
global_churn = df_full_train['churn'].mean()
print(global_churn)

0.26996805111821087


In [22]:
from IPython.display import display

In [23]:
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)
    print()
    print()

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980




seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208




partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730




paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560




paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


## **Feature Importance: Mutual Information**

Mutual Information - concept from Information theory, it tells us how much we can learn about one variable, if we know the value of another

- Higher mutual information means more information about churn from a feature.
- Lower mutual information means less information.
- [More Information on Mutual information](https://en.wikipedia.org/wiki/Mutual_information)

In [24]:
from sklearn.metrics import mutual_info_score

In [25]:
categories = {}
for category in categorical:
    score = mutual_info_score(df_full_train['churn'], df_full_train[category])
    categories[category] = round(float(score), 5)

# DataFrame creation
df_mi = pd.DataFrame.from_dict(
    categories, 
    orient='index', 
    columns=['mutual_info_score']
).sort_values('mutual_info_score', ascending=False).reset_index()

df_mi.columns = ['features', 'mutual_info_score']
df_mi


,features,mutual_info_score
0,contract,0.09832
1,onlinesecurity,0.06309
2,techsupport,0.06103
3,internetservice,0.05587
4,onlinebackup,0.04692
5,deviceprotection,0.04345
6,paymentmethod,0.04321
7,streamingtv,0.03185
8,streamingmovies,0.03158
9,paperlessbilling,0.01759


## **Feature Importance: Correlation**

A way to measure feature importance for numerical variables is using correlation coefficient

Correlation coefficient - https://en.wikipedia.org/wiki/Pearson_correlation_coefficient

- Pearson’s correlation coefficient (r) quantifies the linear dependence between two numeric variables. (Value range: -1 to 1.)

- Negative correlation: If one variable increases, the other tends to decrease (r < 0).

- Positive correlation: Both variables tend to increase together (r > 0).

- Small |r| values (e.g., between ~0.0 to 0.1) indicate very low or no correlation.

- Moderate correlation roughly ~0.2–0.5 (positive or negative).

- Strong correlation if |r| is close to 1, that is, 0.6-1.0.

In [26]:
# check the correlation if the numerical variables with the target variable
df_full_train[numerical].corrwith(df_full_train.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

Correlation results from data:

- Tenure: negative correlation with churn → longer stays lower churn.
- Total charges: also negative → higher total spend lower churn.
- Monthly charges: positive correlation → higher bills associated with higher churn.

## **One-hot Encoding**

- use scikit-learn to encode categorical variables

In [27]:
from sklearn.feature_extraction import DictVectorizer

In [28]:
dv = DictVectorizer(sparse = False)

train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)

## **Logistics Regression**

- Binary Classification
- Linear vs Logistics Regression

In [29]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def linear_regression(xi):
    result = w0

    for j in range(len(xi)):
        result = result + w[j] * xi[j]
    return result

def logistic_regression(xi):
    log_odds = w0

    for j in range(len(xi)):
        log_odds = log_odds + w[j] * xi[j]

    result = sigmoid(log_odds)
    return result

## **Training Logistics Regression with Scikit-Learn**

- Train a model with scikit learn
- Apply it to the validation dataset
- Calculate the accuracy

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
model = LogisticRegression()
model.fit(X_train, y_train)

C:\Users\osaze\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [32]:
model.coef_[0].round(3)

array([ 0.475, -0.175, -0.408, -0.03 , -0.078,  0.063, -0.089, -0.081,
       -0.034, -0.073, -0.335,  0.316, -0.089,  0.004, -0.258,  0.141,
        0.009,  0.063, -0.089, -0.081,  0.266, -0.089, -0.284, -0.231,
        0.123, -0.166,  0.058, -0.087, -0.032,  0.07 , -0.059,  0.141,
       -0.249,  0.215, -0.12 , -0.089,  0.102, -0.071, -0.089,  0.052,
        0.213, -0.089, -0.232, -0.07 ,  0.   ])

In [33]:
print(model.intercept_[0])

-0.1090293966428555


In [34]:
y_pred = model.predict_proba(X_val)[:, 1]

In [35]:
churn_decision = (y_pred >= 0.5)

In [36]:
# let us seperate the clients that will churn
df_val[churn_decision].customerid

3       8433-wxgna
8       3440-jpscl
11      2637-fkfsy
12      7228-omtpn
19      6711-fldfb
           ...    
1397    5976-jcjrh
1398    2034-cgrhz
1399    5276-kqwhg
1407    6521-yytyi
1408    3049-solay
Name: customerid, Length: 311, dtype: object

In [37]:
# let us check how accurate our predictions are

df = pd.DataFrame()
df['probability'] = y_pred
df['prediction'] = churn_decision.astype(int)
df['actual'] = y_val
df['correct'] = df.prediction == df.actual
df

,probability,prediction,actual,correct
0,0.008998,0,0,True
1,0.204548,0,0,True
2,0.212235,0,0,True
3,0.543043,1,1,True
4,0.213792,0,0,True
...,...,...,...,...
1404,0.313687,0,0,True
1405,0.039362,0,1,False
1406,0.136411,0,0,True
1407,0.799764,1,1,True


In [38]:
# let us check the fraction of correct ones

print(df.correct.mean())

0.8034066713981547


## **Model Interpretation**

- Look at the coefficient

In [39]:
dv.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'dependents=no', 'dependents=yes',
       'deviceprotection=no', 'deviceprotection=no_internet_service',
       'deviceprotection=yes', 'gender=female', 'gender=male',
       'internetservice=dsl', 'internetservice=fiber_optic',
       'internetservice=no', 'monthlycharges', 'multiplelines=no',
       'multiplelines=no_phone_service', 'multiplelines=yes',
       'onlinebackup=no', 'onlinebackup=no_internet_service',
       'onlinebackup=yes', 'onlinesecurity=no',
       'onlinesecurity=no_internet_service', 'onlinesecurity=yes',
       'paperlessbilling=no', 'paperlessbilling=yes', 'partner=no',
       'partner=yes', 'paymentmethod=bank_transfer_(automatic)',
       'paymentmethod=credit_card_(automatic)',
       'paymentmethod=electronic_check', 'paymentmethod=mailed_check',
       'phoneservice=no', 'phoneservice=yes', 'seniorcitizen',
       'streamingmovies=no', 'streamingmovies=no_internet_service',

In [40]:
df_coef = pd.DataFrame(model.coef_[0].round(3))
df_coef.index = list(dv.get_feature_names_out())
df_coef = df_coef.reset_index()
df_coef.columns = ['features', 'coefficient']
df_coef.sort_values(by = ['features']).head()

,features,coefficient
0,contract=month-to-month,0.475
1,contract=one_year,-0.175
2,contract=two_year,-0.408
3,dependents=no,-0.030
4,dependents=yes,-0.078


In [41]:
df_coef['|coef|'] = abs(df_coef.coefficient)
df_coef.sort_values(by = ['|coef|'], ascending = False)
df_coef.reset_index(drop = True)

,features,coefficient,|coef|
0,contract=month-to-month,0.475,0.475
1,contract=one_year,-0.175,0.175
2,contract=two_year,-0.408,0.408
3,dependents=no,-0.030,0.030
4,dependents=yes,-0.078,0.078
5,deviceprotection=no,0.063,0.063
6,deviceprotection=no_internet_service,-0.089,0.089
7,deviceprotection=yes,-0.081,0.081
8,gender=female,-0.034,0.034
9,gender=male,-0.073,0.073


**Model Interpretation**

🔴 **Strong influencers (|coef| ≥ 0.30)**

| Feature                     | Coefficient | Impact                |
| --------------------------- |----------- |--------------------- |
| contract=month-to-month     | +0.475      | Highest churn risk    |
| contract=two_year           | -0.408      | Strong retention      |
| internetservice=dsl         | -0.335      | DSL = loyal customers |
| internetservice=fiber_optic | +0.316      | Fiber = churn risk    |

🟡 **Moderate influencers (0.10 ≤ |coef| < 0.30)**

| Feature             | Abs(coef) |
|:------------------- | --------- |
| onlinesecurity=yes  | 0.284     |
| techsupport=yes     | 0.232     |
| seniorcitizen       | 0.215     |
| paperlessbilling=no | 0.231     |

🟢 **Weak influencers (|coef| < 0.10)**

| Feature        | Abs(coef) |
| -------------- | --------- |
| monthlycharges | 0.004     |
| totalcharges   | 0.000     |
| gender=male    | 0.073     |

**Summary**

*feature influence was evaluated using the absolute magnitude of the logistic regression coefficients in log-odds space. The sign of the coefficient indicates direction, while the absolute value reflects the strength of the effect. I classify features with absolute coefficients above 0.3 as strong, between 0.1 and 0.3 as moderate, and below 0.1 as weak.*

**Key Insights:**

- Month-to-month + Fiber = DANGER ZONE (churn magnets)
- 2-year DSL = GOLD (loyal customers)
- Support services = Retention levers
- Demographics = Surprisingly weak!
